<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1718]:
import numpy as np
import pandas as pd

In [ ]:
*It is good practice to display the library version numbers for future reference:*

In [0]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.17.2
Pandas:  0.25.1


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [1035]:
# wrong:
#bikes = pd.read_table('dat/bikeshare.csv', header = None)
#print(bikes.head())
#print()

# wrong:
#bikes = pd.read_table('dat/bikeshare.csv', header = 1)
#print(bikes.head())
#print()

# wrong:
#bikes = pd.read_table('dat/bikeshare.csv', header = 0)
#print(bikes.head())

# ?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [1682]:

bike = pd.read_table("../../DATA/bikeshare.csv" , delimiter=",")
print(bike)

       instant      dteday  season  yr  mnth  hr  holiday  weekday  \
0            1  2011-01-01       1   0     1   0        0        6   
1            2  2011-01-01       1   0     1   1        0        6   
2            3  2011-01-01       1   0     1   2        0        6   
3            4  2011-01-01       1   0     1   3        0        6   
4            5  2011-01-01       1   0     1   4        0        6   
...        ...         ...     ...  ..   ...  ..      ...      ...   
17374    17375  2012-12-31       1   1    12  19        0        1   
17375    17376  2012-12-31       1   1    12  20        0        1   
17376    17377  2012-12-31       1   1    12  21        0        1   
17377    17378  2012-12-31       1   1    12  22        0        1   
17378    17379  2012-12-31       1   1    12  23        0        1   

       workingday  weathersit  temp   atemp   hum  windspeed  casual  \
0               0           1  0.24  0.2879  0.81     0.0000       3   
1              

Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [863]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [864]:
df = pd.read_excel("../../DATA/Iris.xls",  sheet_name = "Data")
df



,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [117]:
pd.read_excel("../../DATA/Iris.xls",   sheet_name = "Data" ,usecols= (1,2,3,4, 5))
df.iloc[: , 1:]

,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,0.2,1.4,3.5,5.1,Setosa
1,0.2,1.4,3.0,4.9,Setosa
2,0.2,1.3,3.2,4.7,Setosa
3,0.2,1.5,3.1,4.6,Setosa
4,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...
145,2.3,5.2,3.0,6.7,Verginica
146,1.9,5.0,2.5,6.3,Verginica
147,2.0,5.2,3.0,6.5,Verginica
148,2.3,5.4,3.4,6.2,Verginica


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [98]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-06-25T04:42:35.700Z,36.438833,-117.963167,6.41,2.67,ml,22.0,129,0.094350,0.18,...,2020-06-25T04:46:20.848Z,"18km NNE of Olancha, CA",earthquake,0.36,1.28,0.144,25,automatic,ci,ci
1,2020-06-25T04:09:17.732Z,38.419000,-112.254400,7.42,2.70,ml,NaN,60,0.243000,0.61,...,2020-06-25T04:48:00.084Z,"3 km SSW of Marysvale, Utah",earthquake,1.30,5.00,0.031,138,reviewed,us,us
2,2020-06-25T04:09:07.460Z,38.775002,-122.743835,0.06,3.42,ml,64.0,54,0.009925,0.09,...,2020-06-25T04:34:03.461Z,"1km ESE of The Geysers, CA",earthquake,0.14,0.26,0.371,16,automatic,nc,nc


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [93]:
url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
df = pd.read_html(url)
df

[                             Bank Name           City  ST   CERT  \
 0                 The First State Bank  Barboursville  WV  14361   
 1                   Ericson State Bank        Ericson  NE  18265   
 2     City National Bank of New Jersey         Newark  NJ  21111   
 3                        Resolute Bank         Maumee  OH  58317   
 4                Louisa Community Bank         Louisa  KY  58112   
 ..                                 ...            ...  ..    ...   
 556                 Superior Bank, FSB       Hinsdale  IL  32646   
 557                Malta National Bank          Malta  OH   6629   
 558    First Alliance Bank & Trust Co.     Manchester  NH  34264   
 559  National State Bank of Metropolis     Metropolis  IL   3815   
 560                   Bank of Honolulu       Honolulu  HI  21029   
 
                    Acquiring Institution       Closing Date  
 0                         MVB Bank, Inc.      April 3, 2020  
 1             Farmers and Merchants Bank  F

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [1683]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [266]:
bike.shape


(17379, 1)

What are the column names in `bikes`?

In [267]:
bike.columns

Index(['instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt'], dtype='object')

What are the data types of these columns?

In [1684]:
bike.dtypes


instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?


In [990]:
bike.index



RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [332]:
bike[['season']]
bike['season']

0        1
1        1
2        1
3        1
4        1
        ..
17374    1
17375    1
17376    1
17377    1
17378    1
Name: season, Length: 17379, dtype: int64

How would we use object notation to show the first 4 rows of `atemp`?

In [911]:
bike.iloc[0:4, 11:12]
bike.loc[:4,'atemp' : 'atemp']

,atemp
0,0.2879
1,0.2727
2,0.2727
3,0.2879
4,0.2879


Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [937]:
#bike
#bike[0:]
bike[0:0]
 


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt


What is the correct way to access the 1st row of the DataFrame by its index?

In [317]:
bike[0:1]


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16


What is the correct way to access the 2nd column of the DataFrame by its index?

In [348]:
#bike['dteday']
bike.iloc[ :, 0:1]


,instant
0,1
1,2
2,3
3,4
4,5
...,...
17374,17375
17375,17376
17376,17377
17377,17378


## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [354]:
bike.isnull()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17375,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17376,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17377,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [940]:
#x = bike.isnull().count() 
bike.isnull().count().sum()

295443

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [941]:
bike.isnull().count()


instant       17379
dteday        17379
season        17379
yr            17379
mnth          17379
hr            17379
holiday       17379
weekday       17379
workingday    17379
weathersit    17379
temp          17379
atemp         17379
hum           17379
windspeed     17379
casual        17379
registered    17379
cnt           17379
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [1693]:
# np.nan  Convert a string or number to a floating point number

x = 2
y = np.nan
x_float = print(x, y)


2 nan


Apply the Pandas `isna` function to the following data objects:

In [872]:
x = 2.3
y = np.nan
print(x, y)




2.3 nan


In [942]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [1707]:
#ANSWER


In [952]:
#How is the pandas I/O parameter `na_values` used?


<bound method NDFrame.get of      Species_No  Petal_width  Petal_length  Sepal_width  Sepal_length  \
0             1          0.2           1.4          3.5           5.1   
1             1          0.2           1.4          3.0           4.9   
2             1          0.2           1.3          3.2           4.7   
3             1          0.2           1.5          3.1           4.6   
4             1          0.2           1.4          3.6           5.0   
..          ...          ...           ...          ...           ...   
145           3          2.3           5.2          3.0           6.7   
146           3          1.9           5.0          2.5           6.3   
147           3          2.0           5.2          3.0           6.5   
148           3          2.3           5.4          3.4           6.2   
149           3          1.8           5.1          3.0           5.9   

    Species_name  
0         Setosa  
1         Setosa  
2         Setosa  
3         Setosa  

? ANSWER: 

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [965]:
bike['season'].max()


4

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [867]:
#ANSWER
bike.iloc[:, [0,1,13]].max()


instant           17379
dteday       2012-12-31
windspeed        0.8507
dtype: object

In [891]:
bike.loc[:, ['instant', 'dteday', 'windspeed']].max()

instant           17379
dteday       2012-12-31
windspeed        0.8507
dtype: object

Compute and print the overall minimum and maximum of the numeric data columns:

In [1069]:
x = bike.select_dtypes(include='int64')
x.max()
#x.min()
#x.dtypes

instant       17379
season            4
yr                1
mnth             12
hr               23
holiday           1
weekday           6
workingday        1
weathersit        4
casual          367
registered      886
cnt             977
dtype: int64

In [970]:
bike

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61


### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [882]:
bike['atemp'].quantile(0.5)

0.4848

0.50    0.4848
0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [0]:
#ANSWER


In [1079]:
bike['atemp'].quantile([0.5, .10, 0.25, 0.50, 0.75 , 0.90])

0.50    0.4848
0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [1710]:
bike['atemp'].describe

<bound method NDFrame.describe of 0        0.2879
1        0.2727
2        0.2727
3        0.2879
4        0.2879
          ...  
17374    0.2576
17375    0.2576
17376    0.2576
17377    0.2727
17378    0.2727
Name: atemp, Length: 17379, dtype: float64>

In [1524]:
bike.sample(5)


,instant,dteday,atemp_level,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
3288,3289,2011-05-21,warm,2,0,5,22,0,6,0,1,0.60,0.5909,0.69,0.1343,78,126,204
4429,4430,2011-07-08,warm,3,0,7,11,0,5,1,2,0.74,0.7121,0.74,0.2537,61,104,165
16926,16927,2012-12-13,mild,4,1,12,1,0,4,1,2,0.28,0.2576,0.52,0.2836,4,19,23
2551,2552,2011-04-21,mild,2,0,4,5,0,4,1,2,0.42,0.4242,0.54,0.5522,0,14,14
7943,7944,2011-12-02,mild,4,0,12,15,0,5,1,1,0.42,0.4242,0.41,0.0896,24,175,199


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [1721]:
import matplotlib.pyplot as plt
atemp_level = pd.cut(bike['atemp'], bins = 4)         


What is `atemp_level`?
Deviding the temp values to 4 groups

In [1712]:
#ANSWER
atemp_level.sample(5)     

7472     (0.25, 0.5]
15159    (0.5, 0.75]
10210    (0.25, 0.5]
7489     (0.5, 0.75]
12126    (0.5, 0.75]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

Here is a random sample of `atemp_level`:

In [1723]:
atemp_level.sample(5)          

15216    (0.25, 0.5]
12207    (0.5, 0.75]
5206     (0.5, 0.75]
6661     (0.25, 0.5]
3746     (0.5, 0.75]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [1728]:
atemp_level = pd.cut(bike['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

1795    mild
5632    warm
3022    warm
4313     hot
8586    mild
Name: atemp, dtype: category
Categories (4, object): [cool < mild < warm < hot]

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [1729]:
#ANSWER
bike['atemp_level'] = atemp_level
(bike['atemp_level']== 'mild').value_counts() 


bike[bike['season'] == 2][bike['atemp_level']== 'mild']     

/Users/doron/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,atemp_level
1807,1808,2011-03-21,2,0,3,0,0,1,1,3,0.34,0.303,0.66,0.3881,2,11,13,mild
1808,1809,2011-03-21,2,0,3,1,0,1,1,2,0.34,0.303,0.71,0.3881,1,6,7,mild
1809,1810,2011-03-21,2,0,3,2,0,1,1,2,0.34,0.303,0.71,0.3284,1,5,6,mild
1810,1811,2011-03-21,2,0,3,3,0,1,1,2,0.34,0.303,0.71,0.3284,0,1,1,mild
1811,1812,2011-03-21,2,0,3,5,0,1,1,1,0.32,0.303,0.81,0.2985,1,1,2,mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12674,12675,2012-06-17,2,1,6,7,0,0,0,1,0.52,0.500,0.72,0.0896,18,30,48,mild
12694,12695,2012-06-18,2,1,6,3,0,1,1,2,0.52,0.500,0.83,0.2239,1,3,4,mild
12696,12697,2012-06-18,2,1,6,5,0,1,1,3,0.52,0.500,0.83,0.1343,1,22,23,mild
12697,12698,2012-06-18,2,1,6,6,0,1,1,3,0.52,0.500,0.88,0.1343,0,37,37,mild


*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [1434]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [1505]:
# Cleaning Data
c[1:2, 1,1]

array([3])

## Load Data

Load rock.csv and clean the dataset.

In [847]:
pd.read_table("../../DATA/rock.csv" , delimiter= ",")

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,NaN,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


In [1391]:

rock= pd.read_table("../../DATA/rock.csv" , delimiter= ",")

rock.dtypes

Song Clean      object
ARTIST CLEAN    object
Release Year    object
COMBINED        object
First?           int64
Year?            int64
PlayCount        int64
F*G              int64
dtype: object

## Check Column Names

Check column names and clean.

In [777]:
rock.columns

Index(['Song Clean', 'ARTIST CLEAN', 'Release Year', 'COMBINED', 'First?',
       'Year?', 'PlayCount', 'F*G'],
      dtype='object')

# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.b

In [778]:
#rock['Release Year']


#rock[rock.loc[:, 'Release Year'] != 'NaN']


rock["Release Year"].replace({np.nan: 0}, inplace=True)
rock

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,0,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [1536]:
rock.dtypes

Song Clean      object
ARTIST CLEAN    object
Release Year    object
COMBINED        object
First            int64
Year             int64
PlayCount        int64
F*G              int64
dtype: object

In [1537]:
### Replacing "Release Year" Datatype to int
#rock["Release Year"].replace({object : int}, inplace=True)
#rock["Release Year"].apply(lambda x: x.dtype)
counter = 0
rock.dtypes
for row in rock['Release Year']:
    counter = counter + 1
  #  print(rock.iloc[:counter, 2]).replace({row : int(row)}


In [1959]:
rock['Release Year'] = pd.to_numeric(rock['Release Year'], downcast='integer', errors='coerce')
rock['Year'] = rock['Release Year'].copy()
#rock.loc[ :, ['Year']]

rock.dtypes


Song Clean       object
ARTIST CLEAN     object
Release Year    float32
COMBINED         object
First             int64
Year            float32
PlayCount         int64
F*G               int64
dtype: object

In [1030]:
#First and Year have a "?"`

#rock['Release Year'] = rock.column.astype('int64')
#rock = pd.read_csv(data, dtype={'Col_B':int64})
#rock['Release Year']
rock.columns

Index(['Song Clean', 'ARTIST CLEAN', 'Release Year', 'COMBINED', 'First',
       'Year', 'PlayCount', 'F*G'],
      dtype='object')

# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

illogical - Year = 0, PlayCount =  0

In [724]:
rock.min()
#rock.max()

Song Clean                     #9 Dream
ARTIST CLEAN                .38 Special
COMBINED        #9 Dream by John Lennon
First                                 1
Year                                  0
PlayCount                             0
F*G                                   0
dtype: object

In [1412]:

rock = rock.rename(columns={'Year?': 'Year', 'First?' : 'First' })
#rock = rock(columns={'Release Year': int()})
rock[rock.loc[:, "Song Clean"] == "#9 Dream"]
#rock.dtypes


,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First,Year,PlayCount,F*G,Year
914,#9 Dream,John Lennon,1974,#9 Dream by John Lennon,1,1,2,2,1974


In [2163]:
#rock.dtypes
rock[[rock.loc[:, 'Year'] > 1972] and (rock.loc[:, 'Year'] < 1980)] 
#rock.loc[:, ['Song Clean','ARTIST CLEAN', 'Release Year'] ] 


,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First,Year,PlayCount,F*G
1,Fantasy Girl,.38 Special,0.0,Fantasy Girl by .38 Special,1,0.0,3,0
4,Art For Arts Sake,10cc,1975.0,Art For Arts Sake by 10cc,1,1975.0,1,1
10,"Baby, Please Don't Go",AC/DC,0.0,"Baby, Please Don't Go by AC/DC",1,0.0,1,0
13,CAN'T STOP ROCK'N'ROLL,AC/DC,0.0,CAN'T STOP ROCK'N'ROLL by AC/DC,1,0.0,5,0
14,Dirty Deeds Done Dirt Cheap,AC/DC,1976.0,Dirty Deeds Done Dirt Cheap by AC/DC,1,1976.0,85,85
...,...,...,...,...,...,...,...,...
2221,My Head's In Mississippi,ZZ Top,0.0,My Head's In Mississippi by ZZ Top,1,0.0,1,0
2222,Party On The Patio,ZZ Top,0.0,Party On The Patio by ZZ Top,1,0.0,14,0
2225,She Loves My Automobile,ZZ Top,0.0,She Loves My Automobile by ZZ Top,1,0.0,1,0
2227,Tush,ZZ Top,1975.0,Tush by ZZ Top,1,1975.0,109,109


# Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [2212]:
raw = rock[2:3]
print(raw.loc[:, ['ARTIST CLEAN','Song Clean', 'Release Year'  ]])    

#if raw[raw.loc[:, 'Year'] > 1980] is True :
    #raw[raw.loc[:, 'Year'] > 1980]
#    print('YES')

#if raw.loc[:, 'Year'] > 1980 is True :
    #raw[raw.loc[:, 'Year'] > 1980]
     #print('YES')

#if (raw.loc[:, 'Year'] > 1980) is True:
#        print("xxx")



  ARTIST CLEAN       Song Clean  Release Year
2  .38 Special  Hold On Loosely        1981.0


## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

## Apply these functions to your dataset

## Describe the new float-only DataFrame.

In [1681]:
#Restart rock
rock= pd.read_table("../../DATA/rock.csv" , delimiter= ",", dtype={'Release Year?': int})
rock["Release Year"].replace({np.nan: 0}, inplace=True)
rock = rock.rename(columns={'Year?': 'Year', 'First?' : 'First' })
rock['Year'] = rock['Release Year'].copy()


rock

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First,Year,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1982,82,82
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1981,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1980,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1975,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,0,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1981,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1975,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1983,1,1




> 
>
>




>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---



